In [29]:
using LowLevelFEM

In [30]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = material("cube")
prob = Problem([mat], type=:General, dim=3, fdim=9)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000980206s, CPU 0.000187s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000173997s, CPU 0.000162s)
Info    : Meshing 3D

Problem("cube1", :General, 3, 9, LowLevelFEM.Material[LowLevelFEM.Material("cube", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 27, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [31]:
L = tensorLaplaceMatrix(prob)

sparse([1, 73, 82, 154, 181, 199, 226, 235, 2, 74  …  162, 171, 180, 189, 198, 207, 216, 225, 234, 243], [1, 1, 1, 1, 1, 1, 1, 1, 2, 2  …  243, 243, 243, 243, 243, 243, 243, 243, 243, 243], [0.16666666666666663, 1.8214596497756474e-17, -3.2526065174565133e-18, 1.734723475976807e-17, -0.04166666666666667, -0.04166666666666666, -0.041666666666666664, -0.041666666666666664, 0.16666666666666663, 1.8214596497756474e-17  …  -0.08333333333333333, -0.08333333333333334, -0.08333333333333334, 8.088148206741863e-17, 1.8431436932253575e-18, -1.1926223897340549e-18, -3.241764495731658e-17, 1.0213184464813452e-16, 3.436920886779049e-17, 1.333333333333333], 243, 243)

In [32]:
ν = mat.ν
trL = traceLaplaceMatrix(prob, coefficient=1 / (1 + ν))

sparse([1, 5, 9, 73, 77, 81, 82, 86, 90, 154  …  216, 217, 221, 225, 226, 230, 234, 235, 239, 243], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  243, 243, 243, 243, 243, 243, 243, 243, 243, 243], [0.12820512820512814, 0.12820512820512814, 0.12820512820512814, 1.3010426069826053e-17, 1.3010426069826053e-17, 1.3010426069826053e-17, -4.87890977618477e-18, -4.87890977618477e-18, -4.87890977618477e-18, 1.474514954580286e-17  …  -2.2876665839444144e-17, 8.304988641238964e-17, 8.304988641238964e-17, 8.304988641238964e-17, 2.656295322589486e-17, 2.656295322589486e-17, 2.656295322589486e-17, 1.0256410256410253, 1.0256410256410253, 1.0256410256410253], 243, 243)

In [33]:
K = L + trL

sparse([1, 5, 9, 73, 77, 81, 82, 86, 90, 154  …  216, 217, 221, 225, 226, 230, 234, 235, 239, 243], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  243, 243, 243, 243, 243, 243, 243, 243, 243, 243], [0.29487179487179477, 0.12820512820512814, 0.12820512820512814, 3.122502256758253e-17, 1.3010426069826053e-17, 1.3010426069826053e-17, -8.131516293641283e-18, -4.87890977618477e-18, -4.87890977618477e-18, 3.209238430557093e-17  …  -5.5294310796760726e-17, 8.304988641238964e-17, 8.304988641238964e-17, 1.8518173106052416e-16, 2.656295322589486e-17, 2.656295322589486e-17, 6.093216209368535e-17, 1.0256410256410253, 1.0256410256410253, 2.3589743589743586], 243, 243)

In [34]:
f0 = load("cube", fx=0)
f = loadVector(prob, [f0])

TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [35]:
K * f

TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [36]:
K \ f

TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [37]:
s0 = tensorField(prob, "top", [0 0 0; 0 0 0; 0 0 1])
supp = displacementConstraint("top", uz=0)
bc = constrainedDoFs(prob, [supp]) * 3
all = 1:prob.non*9
free = setdiff(all, bc)
s = tensorField(prob, "top", [0 0 0; 0 0 0; 0 0 0])
f = -K[:, bc] * DoFs(s0)[bc]
DoFs(s)[free] = K[free, free] \ f[free]
s

TensorField
[0.040719696969697274; 0.0; … ; 0.0; 0.9999999999999998;;]

In [38]:
s0

TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [39]:
f[free]

234-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.4423076923076923
 0.3846153846153846
 0.0
 0.0
 0.0
 0.3846153846153846
 0.0
 0.0
 0.0
 0.8846153846153847

In [40]:
showDoFResults(s0, name="s0")
showDoFResults(s, name="s")
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [28]:
gmsh.finalize()